In [ ]:
import pandas as pd
import torch

TRNSLATION

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the pre-trained translation model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("shihab17/bengali-bn-to-en")
model = AutoModelForSeq2SeqLM.from_pretrained("shihab17/bengali-bn-to-en")

In [ ]:
# Check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Move the model to the appropriate device (GPU if available, otherwise CPU)
model = model.to(device)

def translator(sentence):
    # Tokenize the input sentence and move it to the appropriate device
    inputs = tokenizer.encode(sentence, return_tensors="pt").to(device)

    # Get the translation using the model (which runs on GPU if available)
    translated = model.generate(inputs)

    # Decode the translation into a readable format
    translated_sentence = tokenizer.decode(translated[0], skip_special_tokens=True)

    return translated_sentence

LOAD & PROCESS DATA

In [ ]:
test_df = pd.read_csv('/kaggle/input/dlsprint3/test.csv')
train_df = pd.read_csv('/kaggle/input/dlsprint3/train.csv')

In [ ]:
# Apply the translator function to the 'Problem' column
test_df['Translated_Problem'] = test_df['Problem'].apply(translator)

print(test_df['Translated_Problem'])

In [ ]:
print(test_df['Translated_Problem'])

NUMINA MODEL

In [ ]:
import re
import torch
from transformers import pipeline
import io
import sys


PROMPT ENGINNERING

In [ ]:
# Initialize the pipeline
pipe = pipeline("text-generation", model="AI-MO/NuminaMath-7B-TIR", torch_dtype=torch.bfloat16, device_map="auto")

In [ ]:



def solve_problem(problem):
    messages = [
        {
            "role": "user",
            "content": f"""
            Problem: {problem}
            
            Instruction: Write Python code to solve the problem, ensuring the result is an integer. The code should return only an integer as output.
            Note that the intermediary calculations may be real numbers, but the final numercal answer would always be an integer.do not give 0 as answer.        
            
Example 1:
  Input: Sum of all prime numbers between 50 and 100
  Output: 1060
  Explanation: The prime numbers between 50 and 100 are 53, 59, 61, 67, 71, 73, 79, 83, 89, 97. 
  Their sum is:
  53 + 59 + 61 + 67 + 71 + 73 + 79 + 83 + 89 + 97 = 1060

  Example 2:
  Input: Product of all even numbers between 1 and 20, excluding multiples of 5
  Output: 46080
  Explanation: The even numbers between 1 and 20 are 2, 4, 6, 8, 10, 12, 14, 16, 18, 20. Excluding multiples of 5 (10 and 20), we have:
  2 * 4 * 6 * 8 * 12 * 14 * 16 * 18 = 46080

  Example 3:
  Input: Count of Fibonacci numbers less than 1000
  Output: 16
  Explanation: The Fibonacci sequence is 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987. 
  There are 16 Fibonacci numbers less than 1000.

  Example 4:
  Input: Difference between the sum of the squares of all integers from 1 to 50 and the sum of all integers from 1 to 50
  Output: 42925
  Explanation: The sum of the squares of integers from 1 to 50 is:
  1^2 + 2^2 + 3^2 + ... + 50^2 = 42925. 
  The sum of integers from 1 to 50 is:
  1 + 2 + 3 + ... + 50 = 1275. 
  The difference is:
  42925 - 1275 = 42925 
            """
        },
    ]
    
    # Create the prompt for the model
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    # Configuration for text generation
    gen_config = {
        "max_new_tokens": 1024,
        "do_sample": False,
        "stop_strings": ["```output"],  # Stop at the code output block
        "tokenizer": pipe.tokenizer,
    }

    # Generate the solution using the pipeline
    outputs = pipe(prompt, **gen_config)
    
    # Extract the generated text (solution)
    text = outputs[0]["generated_text"]
    
    #print(text)
    
    # Attempt to extract Python code if present
    python_code_matches = re.findall(r"```python(.*?)```", text, re.DOTALL)
    
    if python_code_matches:
        python_code = python_code_matches[0]
        #print("Extracted Python code:")
        #print(python_code)
        
        # Add validation or manual inspection before executing the code
        try:
            # Redirect the output to capture the result of the print statement
            old_stdout = sys.stdout
            new_stdout = io.StringIO()
            sys.stdout = new_stdout
            
            # Execute the extracted Python code
            exec(python_code)

            # Capture the output
            result = new_stdout.getvalue().strip()

            # Reset stdout
            sys.stdout = old_stdout

            # Extract the final numeric answer from the output
            print(f"Extracted answer: {result}")
            return result

        except Exception as e:
            print(f"Error executing code: {e}")
            return None
    else:
        print("No Python code found in the response.")
        return None



In [ ]:

def solve_problem(problem):
    messages = [
        {
            "role": "user",
            "content": f"""
            Problem: {problem}
            
            Instruction: Write Python code to solve the problem, ensuring the result is an integer. The code should return only an integer as output.
            Note that the intermediary calculations may be real numbers, but the final numercal answer would always be an integer.do not give 0 as answer.        
            
Example 1:
  Input: Sum of all prime numbers between 50 and 100
  Output: 1060
  Explanation: The prime numbers between 50 and 100 are 53, 59, 61, 67, 71, 73, 79, 83, 89, 97. 
  Their sum is:
  53 + 59 + 61 + 67 + 71 + 73 + 79 + 83 + 89 + 97 = 1060

  Example 2:
  Input: Product of all even numbers between 1 and 20, excluding multiples of 5
  Output: 46080
  Explanation: The even numbers between 1 and 20 are 2, 4, 6, 8, 10, 12, 14, 16, 18, 20. Excluding multiples of 5 (10 and 20), we have:
  2 * 4 * 6 * 8 * 12 * 14 * 16 * 18 = 46080
            """
        },
    ]
    
    # Create the prompt for the model
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    # Configuration for text generation
    gen_config = {
        "max_new_tokens": 1024,
        "do_sample": False,
        "stop_strings": ["```output"],  # Stop at the code output block
        "tokenizer": pipe.tokenizer,
    }

    # Generate the solution using the pipeline
    outputs = pipe(prompt, **gen_config)
    
    # Extract the generated text (solution)
    text = outputs[0]["generated_text"]
    
    print(text)
    
    # Attempt to extract Python code if present
    python_code_matches = re.findall(r"```python(.*?)```", text, re.DOTALL)
    
    if python_code_matches:
        python_code = python_code_matches[0]
        #print("Extracted Python code:")
        #print(python_code)
        
        # Add validation or manual inspection before executing the code
        try:
            # Redirect the output to capture the result of the print statement
            old_stdout = sys.stdout
            new_stdout = io.StringIO()
            sys.stdout = new_stdout
            
            # Execute the extracted Python code
            exec(python_code)

            # Capture the output
            result = new_stdout.getvalue().strip()

            # Reset stdout
            sys.stdout = old_stdout

            # Extract the final numeric answer from the output
            print(f"Extracted answer: {result}")
            return result

        except Exception as e:
            print(f"Error executing code: {e}")
            return None
    else:
        print("No Python code found in the response.")
        return None



In [ ]:
test_df['Translated_Problem'][0]

In [ ]:
solve_problem(test_df['Translated_Problem'][0])

Submission

In [ ]:
import csv
from tqdm import tqdm
import gc

In [ ]:
# In DEBUG mode, infer only on 5 problems
DEBUG = False 
# Number of candidate solutions to generate
K = 4

In [ ]:
if DEBUG:
    test_df = test_df[:5]
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
file = open('submission.csv', 'w', encoding='utf-8')
writer = csv.writer(file)
writer.writerow(['ID', 'Answer'])

for row in tqdm(test_df.values):
    id = row[0]
    translated_problem = row[2]    
    answer = solve_problem(translated_problem)
    print(f"solution answer-----{answer}")
    
    if DEBUG:
        print('id: ', id)
        print('problem: ', problem)
        print('answer: ', answer)
        
    # Handling different answer types
    if answer is None:
        answer = 0
    
        
    writer.writerow([id, answer])
    
file.close()

In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv('submission.csv')
df

In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('submission.csv')

# Function to check if a value is an integer or float and return an integer, otherwise return 0
def ensure_integer(value):
    try:
        # If it's a float, round it to the nearest integer
        if isinstance(value, float):
            return round(value)
        # Try converting the value to an integer
        return int(value)
    except (ValueError, TypeError):
        # If conversion fails (e.g., for strings), return 0
        return 0

# Apply the function to the entire DataFrame
df = df.applymap(ensure_integer)

# Display the cleaned DataFrame
print(df)

# Optionally, save the cleaned DataFrame back to a new CSV file
df.to_csv('cleaned_file.csv', index=False)


In [ ]:
# Apply the model to each problem in the DataFrame
test_df['Solution'] = test_df['Translated_Problem'].apply(solve_problem)

# Display the DataFrame with the solutions
print(test_df[['Translated_Problem', 'Solution']])